# 44-霍夫变换（Hough Transform）／直线检测——第一步：霍夫变换

霍夫变换，是将坐标由直角座标系变换到极坐标系，然后再根据数学表达式检测某些形状（如直线和圆）的方法。当直线上的点变换到极座标中的时候，会交于一定的$r$、$t$的点。这个点即为要检测的直线的参数。通过对这个参数进行逆变换，我们就可以求出直线方程。

方法如下：

1. 用边缘图像来对边缘像素进行霍夫变换。
2. 在霍夫变换后获取值的直方图并选择最大点。
3. 对极大点的r和t的值进行霍夫逆变换以获得检测到的直线的参数。

在这里，进行一次霍夫变换之后，可以获得直方图。算法如下：

1. 求出图像的对角线长$r_{max}$；
2. 在边缘点$(x,y)$处，$t$取遍$[0,179]$，根据下式执行霍夫变换： $$ r_{ho}=x\ \cos(t)+y\ \sin(t) $$
3. 做一个$180\times r_{max}$大小的表，将每次按上式计算得到的表格$(t,r)$处的值加1。换句话说，这就是在进行投票。票数会在一定的地方集中。

对图片来计算投票之后的表。使用如下参数进行 Canny 边缘检测：高斯滤波器$(5\times5,s = 1.4)$，$HT = 100$，$LT = 30$。

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
img = cv2.imread("../picture/chans.png").astype(np.float32)

In [3]:
# 灰度化
def BGR2GRAY(img):
    
    b = img[:,:,0]
    g = img[:,:,1]
    r = img[:,:,2]
    
    gray = 0.2126 * r + 0.7152 * g + 0.0722 * b
    gray = gray.astype(np.uint8)
    
    return gray

In [4]:
def Gaussian_filter(img, K_size, sigma):
    
    if len(img.shape) == 3:
        H, W, C = img.shape
        gray = False
    else:
        img = np.expand_dims(img, axis=-1)
        H, W, C = img.shape
        gray = True
        
    # Zero padding
    pad = K_size // 2
    out = np.zeros([H + pad * 2, W + pad * 2, C], dtype=np.float)
    out[pad: pad + H, pad: pad + W] = img.copy().astype(np.float)

    # prepare Kernel
    K = np.zeros((K_size, K_size), dtype=np.float)
    for x in range(-pad, -pad + K_size):
        for y in range(-pad, -pad + K_size):
            K[y + pad, x + pad] = np.exp( - (x ** 2 + y ** 2) / (2 * (sigma ** 2)))
    K /= (2 * np.pi * sigma * sigma)
    K /= K.sum()

    tmp = out.copy()

    # filtering
    for y in range(H):
        for x in range(W):
            for c in range(C):
                out[pad + y, pad + x, c] = np.sum(K * tmp[y : y + K_size, x : x + K_size, c])
                
    out = np.clip(out, 0, 255)
    out = out[pad : pad + H, pad : pad + W]

    if gray:
        out = out[..., 0]

    return out

In [5]:
# Sobel
def Sobel_filter(img, K_size):
    
    if len(img.shape) == 3:
        H, W, C = img.shape
    else:
        H, W = img.shape

    # Zero padding
    pad = K_size // 2
    out = np.zeros((H + pad * 2, W + pad * 2), dtype=np.float)
    out[pad : pad + H, pad : pad + W] = img.copy().astype(np.float)

    tmp = out.copy()
    out_v = out.copy()
    out_h = out.copy()
    
    # Sobel vertical
    Kv = [[1., 2., 1.],[0., 0., 0.], [-1., -2., -1.]]
    # Sobel horizontal
    Kh = [[1., 0., -1.],[2., 0., -2.],[1., 0., -1.]]

    # filtering
    for y in range(H):
        for x in range(W):
            out_v[pad + y, pad + x] = np.sum(Kv * tmp[y : y + K_size, x : x + K_size])
            out_h[pad + y, pad + x] = np.sum(Kh * tmp[y : y + K_size, x : x + K_size])
                
    out_v = np.clip(out_v, 0, 255)
    out_h = np.clip(out_h, 0, 255)
    
    out_v = out_v[pad : pad + H, pad : pad + W].astype(np.uint8)
    out_h = out_h[pad : pad + H, pad : pad + W].astype(np.uint8)

    return out_v, out_h

In [6]:
# get edge strength, angle
def get_edge_angle(fx, fy):
    
    edge = np.sqrt(np.power(fx, 2) + np.power(fy, 2))
    fx = np.maximum(fx, 1e-5)
    
    angle = np.arctan(fy / fx)
    
    return edge, angle

def angle_quantization(angle):
    
    angle = angle / np.pi * 180
    
    angle[angle < -22.5] = 180 + angle[angle < -22.5]
    _angle = np.zeros_like(angle, dtype=np.uint8)
    _angle[np.where(angle <= 22.5)] = 0
    _angle[np.where((angle > 22.5) & (angle <= 67.5))] = 45
    _angle[np.where((angle > 67.5) & (angle <= 112.5))] = 90
    _angle[np.where((angle > 112.5) & (angle <= 157.5))] = 135
    
    return _angle

In [7]:
# 非极大值抑制
def non_maximum_suppression(angle, edge):
    
    H, W = angle.shape
    _edge = edge.copy()
    
    for y in range(H):
        for x in range(W):
            if angle[y, x] == 0:
                dx1, dy1, dx2, dy2 = -1, 0, 1, 0
            elif angle[y, x] == 45:
                dx1, dy1, dx2, dy2 = -1, 1, 1, -1
            elif angle[y, x] == 90:
                dx1, dy1, dx2, dy2 = 0, -1, 0, 1
            elif angle[y, x] == 135:
                dx1, dy1, dx2, dy2 = -1, -1, 1, 1
            
            if x == 0:
                dx1 = max(dx1, 0)
                dx2 = max(dx2, 0)
            if x == W-1:
                dx1 = min(dx1, 0)
                dx2 = min(dx2, 0)
            if y == 0:
                dy1 = max(dy1, 0)
                dy2 = max(dy2, 0)
            if y == H-1:
                dy1 = min(dy1, 0)
                dy2 = min(dy2, 0)
                
            if max(max(edge[y, x], edge[y + dy1, x + dx1]), edge[y + dy2, x + dx2]) != edge[y, x]:_edge[y, x] = 0
    
    return _edge

In [8]:
# 阈值
def hysterisis(edge, HT, LT):
    
    H, W = edge.shape
    
    edge[edge >= HT] = 255
    edge[edge <= LT] = 0
    
    _edge = np.zeros((H + 2, W + 2), dtype=np.float32)
    _edge[1 : H + 1, 1 : W + 1] = edge
    
    # 8邻域
    nn = np.array(((1., 1., 1.), (1., 0., 1.), (1., 1., 1.)), dtype=np.float32)
    
    for y in range(1, H+2):
        for x in range(1, W+2):
            if _edge[y, x] < LT or _edge[y, x] > HT:
                continue
            if np.max(_edge[y-1:y+2, x-1:x+2] * nn) >= HT:
                _edge[y, x] = 255
            else:
                _edge[y, x] = 0
                
    edge = _edge[1:H+1, 1:W+1]
    
    return edge

In [9]:
def Canny(img):
    
    def edge_angle(fx, fy):
        
        # get edge strength
        edge = np.sqrt(np.power(fx.astype(np.float32), 2) + np.power(fy.astype(np.float32), 2))
        edge = np.clip(edge, 0, 255)
        
        fx = np.maximum(fx, 1e-5)
        
        angle = np.arctan(fy / fx)
        
        return edge, angle
    
    # grayscale
    gray = BGR2GRAY(img)

    # gaussian filtering
    gaussian = Gaussian_filter(gray, K_size=5, sigma=1.4)

    # sobel filtering
    fy, fx = Sobel_filter(gaussian, K_size=3)

    # get edge strength, angle
    edge, angle = edge_angle(fx, fy)

    # angle quantization
    angle = angle_quantization(angle)
    
    # non maximum suppression
    edge = non_maximum_suppression(angle, edge)
    
    # hysterisis threshold
    out = hysterisis(edge, 50, 20)

    return out

In [13]:
def Hough_Line_step1(edge):
    
    # voting
    H, W = edge.shape
    drho = 1
    dtheta = 1
    
    rho_max = np.ceil(np.sqrt(H ** 2 + W ** 2)).astype(np.int)
    
    # label
    hough = np.zeros([rho_max * 2, 180], dtype=np.int)
    
    index = np.where(edge==255)
    
    # transform
    for y,x in zip(index[0], index[1]):
        for theta in range(0, 180, dtheta):
            t = np.pi / 180 * theta
            rho = int(x * np.cos(t) + y * np.sin(t))
            
            # vote
            hough[rho + rho_max, theta] += 1
            
    hough = hough.astype(np.uint8)
    
    return hough

In [11]:
edge = Canny(img)

In [14]:
hough = Hough_Line_step1(edge)

In [15]:
hough = hough.astype(np.uint8)

In [16]:
cv2.imwrite('../picture/chan_result44_hough.jpg', hough)
# cv2.namedWindow("result", 0);
# cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", hough)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 45-霍夫变换（Hough Transform）／直线检测——第二步：NMS

第2步

在问题44​获得的表格中，在某个地方附近集中了很多票。这里，执行提取局部最大值的操作。

提取出投票前十名的位置，并将其表示出来

NMS 的算法如下：

1. 在该表中，如果遍历到的像素的投票数大于其8近邻的像素值，则它不变
2. 如果遍历到的像素的投票数小于其8近邻的像素值，则设置为0

In [33]:
def nonmaximum(hough):
    
    rho_max, _ = hough.shape
    
    for y in range(rho_max):
        for x in range(180):
            # 8邻域
            x1 = max(x-1, 0)
            x2 = min(x+2, 180)
            y1 = max(y-1, 0)
            y2 = min(y+2, rho_max-1)
            if np.max(hough[y1:y2, x1:x2]) == hough[y,x] and hough[y, x] != 0:
                pass
            else:
                hough[y,x] = 0
    
    index_x = np.argsort(hough.ravel())[::-1][:20]
    index_y = index_x.copy()
    thetas = index_x % 180
    rhos = index_y // 180
    _hough = np.zeros_like(hough, dtype=np.int)
    _hough[rhos, thetas] = 255
            
    return _hough

In [34]:
def Hough_Line_step2(edge):
    
    hough = Hough_Line_step1(edge)
    
    out = nonmaximum(hough)
    
    return out

In [35]:
nms = Hough_Line_step2(edge)

In [36]:
nms = nms.astype(np.uint8)

In [37]:
cv2.imwrite('../picture/chan_result45_nms.jpg', nms)
# cv2.namedWindow("result", 0);
# cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", nms)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 46-霍夫变换（Hough Transform）／直线检测——第三步：霍夫逆变换

将问题45中得到的极大值进行霍夫逆变换之后画出得到的直线。在这里，已经通过霍夫变换检测出了直线。

算法如下：

1. 极大值点(r,t)通过下式进行逆变换： $$ y = - \frac{\cos(t)}{\sin(t) } \ x + \frac{r}{\sin(t)}\ x = - \frac{\sin(t)}{\cos(t) } \ y + \frac{r}{\cos(t)} $$

2. 对于每个局部最大点，使$y = 0-H -1$，$x = 0-W -1$，然后执行1中的逆变换，并在输入图像中绘制检测到的直线。请将线的颜色设置为红色$(R,G,B) = (255, 0, 0)$。

In [39]:
def non_maximum(hough):
    
    rho_max, _ = hough.shape
    
    for y in range(rho_max):
        for x in range(180):
            # 8邻域
            x1 = max(x-1, 0)
            x2 = min(x+2, 180)
            y1 = max(y-1, 0)
            y2 = min(y+2, rho_max-1)
            if np.max(hough[y1:y2, x1:x2]) == hough[y,x] and hough[y, x] != 0:
                pass
            else:
                hough[y,x] = 0
            
    return hough

In [40]:
def inverse_hough(hough, img):
    
    H, W, C = img.shape
    rho_max, _ = hough.shape
    
    out = img.copy()
    
    index_x = np.argsort(hough.ravel())[::-1][:20]
    index_y = index_x.copy()
    thetas = index_x % 180
    rhos = index_y // 180 - rho_max / 2
    
    for theta, rho in zip(thetas,rhos):
        t = np.pi / 180. * theta
        
        for x in range(W):
            if np.sin(t) != 0:
                y = - (np.cos(t) / np.sin(t)) * x + (rho) / np.sin(t)
                y = int(y)
                if y >= H or y < 0:
                    continue
                out[y, x] = [0, 0, 255]

        for y in range(H):
            if np.cos(t) != 0:
                y = - (np.sin(t) / np.cos(t)) * x + (rho) / np.cos(t)
                y = int(y)
                if y >= W or y < 0:
                    continue
                out[y, x] = [0, 0, 255]
            
    out = out.astype(np.uint8)
    
    return out

In [41]:
def Hough_Line(edge, img):
    
    hough = Hough_Line_step1(edge)
    
    # non maximum suppression
    hough = non_maximum(hough)

    # inverse hough
    out = inverse_hough(hough, img)
    
    return out

In [42]:
hough = Hough_Line(edge, img)

hough = hough.astype(np.uint8)

In [43]:
cv2.imwrite('../picture/chan_result46_inverse_hough.jpg', hough)
# cv2.namedWindow("result", 0);
# cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", hough)
cv2.waitKey(0)
cv2.destroyAllWindows()